In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchaudio tqdm ffmpeg transformers datasets peft accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01
  Preparing metadata (setup.py) ... done


In [2]:
!pip install gdown

### Data chưa qua lọc nhiễu

In [3]:
!gdown "https://drive.google.com/uc?id=1tywlQSElm8G23MmUSjAtsTw2MKUZd6Qg" -O data.zip

Downloading...
From (original): https://drive.google.com/uc?id=1tywlQSElm8G23MmUSjAtsTw2MKUZd6Qg
From (redirected): https://drive.google.com/uc?id=1tywlQSElm8G23MmUSjAtsTw2MKUZd6Qg&confirm=t&uuid=8b4fc3ee-f3d6-4bea-93bd-4c880121554b
To: /kaggle/working/data.zip
100%|█████████████████████████████████████████| 533M/533M [00:04<00:00, 125MB/s]


In [4]:
!unzip /kaggle/working/data.zip -d /kaggle/working

Archive:  /kaggle/working/data.zip
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047364-1.txt  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047167-1.wav  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047389-1.txt  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047314-1.wav  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047948-1.txt  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000048335-1.txt  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000046538-1.wav  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000046558-1.wav  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047976-1.wav  
  inflating: /kaggle/working/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000047354-1.txt  
  inflating: /kaggle/work

### Data đã lọc nhiễu

In [3]:
!gdown "https://drive.google.com/uc?id=1bOhkqT7iRLZQzjbBwHhZcj8t71ic7JzI" -O data.zip

Downloading...
From (original): https://drive.google.com/uc?id=1bOhkqT7iRLZQzjbBwHhZcj8t71ic7JzI
From (redirected): https://drive.google.com/uc?id=1bOhkqT7iRLZQzjbBwHhZcj8t71ic7JzI&confirm=t&uuid=790b927c-f020-4381-9498-3571a90cf0f7
To: /kaggle/working/data.zip
100%|████████████████████████████████████████| 533M/533M [00:05<00:00, 91.1MB/s]


In [4]:
!unzip /kaggle/working/data.zip -d /kaggle/working

Archive:  /kaggle/working/data.zip
   creating: /kaggle/working/Datasets/
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000005-1.txt  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000005-1.wav  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000028-1.txt  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000028-1.wav  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000033-1.txt  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000033-1.wav  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000045-1.txt  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000045-1.wav  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000000072-1.txt  
  inflating: /kaggle/working/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_00000

In [5]:
DATA_DIR = "/kaggle/working/Datasets"  # Thư mục chứa file .wav/.mp3 và .txt (transcripts)
MODEL_DIR = "/kaggle/working"  # Thư mục lưu mô hình

In [6]:
import os
import torch
import whisper
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.optim import AdamW

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🔹 Using device:", device)

🔹 Using device: cuda


In [8]:
model_name = "openai/whisper-small"
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)
processor = WhisperProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [10]:
lora_config = LoraConfig(
    r=16,  # Rank của LoRA (giá trị nhỏ hơn giúp tiết kiệm tài nguyên)
    lora_alpha=32,  # Hệ số scale
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"],  # Các module cần áp dụng LoRA
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7267


### tải lên lại tham số LoRA đã huấn luyện (Đã lọc nhiễu)

In [9]:
!gdown "https://drive.google.com/uc?id=1Q521ykPZQ057kp7NZS4DXWddG8FM_SNw" -O models.zip

Downloading...
From: https://drive.google.com/uc?id=1Q521ykPZQ057kp7NZS4DXWddG8FM_SNw
To: /kaggle/working/models.zip
100%|██████████████████████████████████████| 13.1M/13.1M [00:00<00:00, 50.2MB/s]


### tải lên lại tham số LoRA đã huấn luyện (Chưa lọc nhiễu)

In [11]:
!gdown "https://drive.google.com/uc?id=1rtgZSIARDrNXvfPXgVNtI2td8awNrE8M" -O models.zip

Downloading...
From: https://drive.google.com/uc?id=1rtgZSIARDrNXvfPXgVNtI2td8awNrE8M
To: /kaggle/working/models.zip
100%|███████████████████████████████████████| 19.7M/19.7M [00:00<00:00, 160MB/s]


In [10]:
!unzip /kaggle/working/models.zip -d /kaggle/working

Archive:  /kaggle/working/models.zip
   creating: /kaggle/working/models/
  inflating: /kaggle/working/models/adapter_config.json  
  inflating: /kaggle/working/models/adapter_model.safetensors  


In [11]:
!pip install safetensors

In [13]:
from safetensors.torch import load_file as safe_load_file

# from peft import PeftModel, LoraConfig

# adapter_model_path = '/kaggle/working/models'
# model = PeftModel.from_pretrained(model, adapter_model_path)
# model.train()
# model.to(device)

# model.print_trainable_parameters()

# Recreate the LoRA config with the same settings as before
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"],
)

# Insert the LoRA adapter into the base model
model = get_peft_model(model, lora_config)

# Load the adapter weights manually from your saved file;
# set strict=False to allow for slight mismatches in keys.
adapter_weights = safe_load_file("/kaggle/working/models/adapter_model.safetensors", device=device)
model.load_state_dict(adapter_weights, strict=False)

model.train()
model.to(device)
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7267


### xong

In [14]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import WhisperFeatureExtractor, WhisperTokenizer
import librosa
from torch.nn.utils.rnn import pad_sequence

# Giả sử bạn đã nạp WhisperFeatureExtractor và WhisperTokenizer
# FEATURE_EXTRACTOR = WhisperFeatureExtractor.from_pretrained('openai/whisper-large')
# TOKENIZER = WhisperTokenizer.from_pretrained('openai/whisper-large', language="vi", task="transcribe")

# Định nghĩa Dataset class
class SpeechDataset(Dataset):
    def __init__(self, audio_files, transcripts, whisper, processor, max_length=448, sampling_rate=16000):
        self.audio_files = audio_files
        self.transcripts = transcripts
        self.whisper = whisper
        self.processor = processor
        self.max_length = max_length
        self.sampling_rate = sampling_rate

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_path = self.audio_files[idx]
        file_name = os.path.basename(audio_path).split(".")[0]
        transcript = self.transcripts.get(file_name, "")

        # Load audio
        audio = self.whisper.load_audio(audio_path)
        inputs = self.processor(audio, return_tensors="pt", sampling_rate=16000)
        # Chuyển đổi âm thanh thành đặc trưng (features)
        input_features = inputs.input_features.squeeze(0)  # Remove batch dimension
        
        # Tokenize transcript
        labels = self.processor.tokenizer(
            transcript, 
            return_tensors="pt",
            truncation=True,
            padding='max_length',  # Padding labels to max_length
            max_length=self.max_length
        ).input_ids.squeeze(0)  # Remove batch dimension

        return {
            "input_features": input_features,  # Không cần tensor cho các input_features vì đã chuẩn bị sẵn
            "labels": labels
        }

# Hàm load dữ liệu
def load_data(data_dir):
    audio_files = []
    transcripts = {}

    for file in os.listdir(data_dir):
        if file.endswith(".wav") or file.endswith(".mp3"):
            audio_files.append(os.path.join(data_dir, file))
        elif file.endswith(".txt"):
            with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
                transcripts[file.replace(".txt", "")] = f.read().strip()

    return audio_files, transcripts

# Padding các batches và chuyển thành tensor
def collate_fn(batch):
    input_features = [item['input_features'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Pad input features to have the same shape
    input_features = pad_sequence(input_features, batch_first=True, padding_value=0)

    # Pad labels to the same length (max_length=448)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # Use -100 for padding labels (ignore in loss)

    return {
        "input_features": input_features,
        "labels": labels
    }

# Sử dụng DataLoader
# DATA_DIR = "/kaggle/working/Data_Goc"  # Đường dẫn tới thư mục chứa dữ liệu
DATA_DIR = "/kaggle/working/Datasets"
audio_files, transcripts = load_data(DATA_DIR)

# Khởi tạo dataset và dataloader
batch_size = 8  # Ví dụ batch size
dataset = SpeechDataset(audio_files, transcripts, whisper, processor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Kiểm tra batch size
for batch in data_loader:
    print(batch["input_features"].shape)  # Kiểm tra kích thước của batch input_features
    print(batch["labels"].shape)  # Kiểm tra kích thước của batch labels
    break

torch.Size([8, 80, 3000])
torch.Size([8, 448])


In [15]:
# Kiểm tra số lượng phần tử trong Dataset và DataLoader
print(f"Total number of samples in dataset: {len(dataset)}")  # Số lượng mẫu trong dataset

# Kiểm tra số lượng batches trong DataLoader
print(f"Total number of batches in DataLoader: {len(data_loader)}")  # Số lượng batch trong DataLoader

# Kiểm tra số lượng phần tử trong batch
for batch in data_loader:
    print(f"Batch input_features shape: {batch['input_features'].shape}")  # Kiểm tra kích thước của input_features trong batch
    print(f"Batch labels shape: {batch['labels'].shape}")  # Kiểm tra kích thước của labels trong batch
    break  # Chỉ in kết quả của batch đầu tiên

Total number of samples in dataset: 2000
Total number of batches in DataLoader: 250
Batch input_features shape: torch.Size([8, 80, 3000])
Batch labels shape: torch.Size([8, 448])


In [ ]:
import torch
from transformers import AdamW
import shutil
import os
import time

# Giả sử 'model' lúc này đã là PeftModel (LoRA) sau khi gọi get_peft_model(...)
optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 50  # số epoch tối đa (đặt ví dụ 50)
patience = 3     # số epoch cho phép không cải thiện
best_loss = float('inf')
best_model_dir = None  # Lưu đường dẫn thư mục "model tốt nhất" hiện tại
epochs_without_improvement = 0

for epoch in range(num_epochs):
    start_time = time.time()
    print(f"===== EPOCH {epoch+1}/{num_epochs} =====")

    running_loss = 0.0
    num_steps = 0
    num_files = 0
    # Vòng lặp qua danh sách file audio từ DataLoader (batch)
    for batch in data_loader:
        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        # Tính loss
        loss = model(input_features, labels=labels, use_cache=False).loss

        # Backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Đếm số bước huấn luyện và loss tổng
        running_loss += loss.item()
        num_steps += 1
        num_files += len(input_features)

        if num_files % 50 == 0:
            print(f"Training in progress: {(num_files / len(audio_files)) * 100:.2f}%")

    # Tính loss trung bình cho epoch
    epoch_loss = running_loss / max(num_steps, 1)
    print(f"Epoch {epoch+1} completed. Average loss: {epoch_loss:.4f}")

    epoch_time = time.time() - start_time
    print(f"Time taken for epoch {epoch+1}: {epoch_time:.2f} seconds")

    # Lưu PEFT model và optimizer state
    tmp_model_dir = f"whisper_lora_epoch_{epoch+1}"
    os.makedirs(tmp_model_dir, exist_ok=True)

    # Lưu PEFT model
    model.save_pretrained(tmp_model_dir)
    # Lưu optimizer state nếu cần khôi phục training
    torch.save(optimizer.state_dict(), os.path.join(tmp_model_dir, "optimizer_state.pt"))
    print(f"Model (LoRA) saved to {tmp_model_dir}")

    # So sánh với best_loss
    if epoch_loss < best_loss:
        # Model này tốt hơn model cũ
        # => xóa model cũ (nếu có)
        if best_model_dir and os.path.exists(best_model_dir):
            # shutil.rmtree(best_model_dir)
            print(f"Removed older best model at {best_model_dir}")

        # Cập nhật best_loss và best_model_dir
        best_loss = epoch_loss
        best_model_dir = tmp_model_dir
        epochs_without_improvement = 0
        print("New best model found!")
    else:
        # Model hiện tại không tốt hơn
        # => Xóa thư mục tạm này đi
        shutil.rmtree(tmp_model_dir)
        print(f"Removed current model at {tmp_model_dir} (not better than best).")

        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"No improvement in {patience} consecutive epochs. Stopping early.")
            break

print("Training has ended.")
print(f"Best model is in: {best_model_dir}, with loss = {best_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


===== EPOCH 1/50 =====
Training in progress: 10.00%
Training in progress: 20.00%
Training in progress: 30.00%
Training in progress: 40.00%
Training in progress: 50.00%
Training in progress: 60.00%
Training in progress: 70.00%
Training in progress: 80.00%
Training in progress: 90.00%
Training in progress: 100.00%
Epoch 1 completed. Average loss: 0.4580
Time taken for epoch 1: 821.86 seconds
Model (LoRA) saved to whisper_lora_epoch_1
New best model found!
===== EPOCH 2/50 =====
Training in progress: 10.00%
Training in progress: 20.00%
Training in progress: 30.00%
Training in progress: 40.00%


In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 1  # Số epoch tối đa
patience = 3     # Số epoch cho phép không cải thiện
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    print(f"===== EPOCH {epoch+1}/{num_epochs} =====")

    running_loss = 0.0
    num_steps = 0

    for audio_path in audio_files:
        file_name = os.path.basename(audio_path).split(".")[0]
        if file_name not in transcripts:
            continue  # Bỏ qua audio nếu không có transcript

        # Load và xử lý audio
        audio = whisper.load_audio(audio_path)
        inputs = processor(audio, return_tensors="pt", sampling_rate=16000)
        input_features = inputs.input_features.to(device)

        # Tokenize transcript (truncation để tránh quá độ dài)
        labels = processor.tokenizer(
            transcripts[file_name], 
            return_tensors="pt",
            truncation=True,
            max_length=448
        ).input_ids.to(device)

        # Tính loss
        loss = model(input_features, labels=labels).loss

        # Backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"- Loss for {file_name}: {loss.item()}")

        # Tính tổng loss
        running_loss += loss.item()
        num_steps += 1

    # Tính loss trung bình cho epoch
    epoch_loss = running_loss / max(num_steps, 1)
    print(f"Epoch {epoch+1} completed. Average loss: {epoch_loss:.4f}")

    # ===== LƯU MÔ HÌNH THEO ĐỊNH DẠNG PEFT =====
    # Tạo thư mục riêng cho mỗi epoch (nếu muốn theo dõi từng epoch)
    model_save_dir = f"whisper_lora_epoch_{epoch+1}"
    os.makedirs(model_save_dir, exist_ok=True)

    # Lưu model: hàm này sẽ lưu cả adapter_config.json và adapter_model.bin
    model.save_pretrained(model_save_dir)
    print(f"Model (LoRA) saved to {model_save_dir}")

    torch.save(optimizer.state_dict(), f"optimizer_epoch_{epoch+1}.pt")

    # ===== EARLY STOPPING =====
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"No improvement in {patience} consecutive epochs. Stopping early.")
            break

print("Training has ended.")

In [7]:
!rm -rf /kaggle/working/dataset/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000018745-1.wav
!rm -rf /kaggle/working/dataset/Datasets/database_sa1_Jan08_Mar19_cleaned_utt_0000018745-1.txt

In [ ]:
import torch
import os
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel, LoraConfig
from torch.optim import AdamW
import whisper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# B1: Load base model và processor như ban đầu
base_model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(base_model_name)
base_whisper = WhisperForConditionalGeneration.from_pretrained(base_model_name)
base_whisper.to(device)

# B2: Load lại trọng số LoRA (đã huấn luyện dở)
checkpoint_dir = "whisper_lora_epoch_10"  # nơi bạn đã save_pretrained() ở epoch 10
model = PeftModel.from_pretrained(base_whisper, checkpoint_dir)
model.to(device)
model.train()  # Bật chế độ train để tiếp tục huấn luyện

# B3: Tạo lại optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# B4 (tuỳ chọn): Load lại state optimizer nếu đã lưu
optimizer_ckpt_path = "optimizer_epoch_10.pt"
if os.path.exists(optimizer_ckpt_path):
    optimizer_state = torch.load(optimizer_ckpt_path, map_location=device)
    optimizer.load_state_dict(optimizer_state)
    print("Optimizer state loaded.")

# B5: Tiếp tục huấn luyện
num_epochs = 50
start_epoch = 10  # hoặc epoch_đang_dừng
patience = 3
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(start_epoch, num_epochs):
    print(f"===== EPOCH {epoch+1}/{num_epochs} =====")

    running_loss = 0.0
    num_steps = 0

    for audio_path in audio_files:
        file_name = os.path.basename(audio_path).split(".")[0]
        if file_name not in transcripts:
            continue

        # Load audio
        audio = whisper.load_audio(audio_path)
        inputs = processor(audio, return_tensors="pt", sampling_rate=16000)
        input_features = inputs.input_features.to(device)

        # Tokenize transcript
        labels = processor.tokenizer(
            transcripts[file_name], 
            return_tensors="pt",
            truncation=True,
            max_length=448
        ).input_ids.to(device)

        # Forward
        loss = model(input_features, labels=labels).loss

        # Backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
        num_steps += 1

    # Tính loss trung bình epoch
    epoch_loss = running_loss / max(num_steps, 1)
    print(f"Epoch {epoch+1} completed. Average loss: {epoch_loss:.4f}")

    # Lưu mô hình + optimizer
    model_save_dir = f"whisper_lora_epoch_{epoch+1}"
    os.makedirs(model_save_dir, exist_ok=True)
    model.save_pretrained(model_save_dir)
    print(f"Model saved to {model_save_dir}")

    torch.save(optimizer.state_dict(), f"optimizer_epoch_{epoch+1}.pt")

    # Early stopping (nếu muốn)
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"No improvement in {patience} consecutive epochs. Stopping early.")
            break
